In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import ipywidgets as widgets
from IPython.display import display

In [98]:
#Conversions
conversions = {
    "kWh_MWh": 1/10**3,
    "kWh_GWh": 1/10**6,
    "kWh_TWh": 1/10**9,
    "Mwh_TWh": 1/10**6,
    "g_kt": 1/10**9,
    "kg_t": 1/10**3,
    "t_kt": 1/10**3,
    "t_Mt": 1/10**6,
    "Mt_kt": 10**3,
    "km_Tkm": 1/10**12,
    "Mv_v": 10**6
}

In [129]:
#Total Metals
df_Metals = pd.read_excel('Metals.xlsx',sheet_name='Metals')
df_Power_Plants = pd.read_excel('Metals.xlsx',sheet_name='Power Systems')
df_Vehicles = pd.read_excel('Metals.xlsx',sheet_name='Vehicles')

In [139]:
#Current World Inputs
'''
These are inputs that I have found but will be subject to change over time
'''
# Create GHG DataFrame
# The sectors and their respective GHG contributions are defined here.
GHG_data = {
    'Sector': ['Oil','Gas','Coal','Biomass','Industry','Waste','Agriculture, Forestry and Land Use', 'Other'],
    'Percentage': [23.2, 16.3, 33.1, 0.6, 5.2, 3.2, 18.4, 100 - 23.2 - 16.3 - 33.1 - 0.6 - 5.2 - 3.2 - 18.4]
}
GHG_df = pd.DataFrame(GHG_data)

# Create Power DataFrame
# The power sources and their respective contributions are defined here.
Power_data = {
    'Source': ['Oil','Gas','Coal','Nuclear','Hydro','Renewables','Other'],
    'Percentage': [2.5, 22.9, 36.0, 9.8, 15.0, 12.8, 100 - 2.5 - 22.9 - 36.0 - 9.8 - 15.0 - 12.8]
}
Power_df = pd.DataFrame(Power_data)

# World values for GHG Emissions and Power Generation
World_GHG_Emissions = 49758 #Mt/y CO2eq - as of 2020
World_Power_Generation = 28466 #TWh - total power produced per year in TWh

# Compute the absolute emissions and power based on the world values
GHG_df['Absolute'] = GHG_df['Percentage']/100 * World_GHG_Emissions
Power_df['Absolute'] = Power_df['Percentage']/100 * World_Power_Generation

# Compute the sum of power (in TWh) from Oil, Gas, and Coal
GHG_Power_TWh = Power_df['Absolute'].loc[Power_df['Source'].isin(['Oil','Gas','Coal'])].sum()

# Create a DataFrame to store this information
power_requirements = pd.DataFrame(
    {'Non-Renewable Power': [GHG_Power_TWh]},
    index=['GHG Power'])
#Other Power Based Inputs
# Constants that will be used in the model
Power_to_Remove_GHG = 0.51 #TWh/Mt CO2eq
Current_Year = 2023
Years_To_Develop = 10 #Time in years to develop a mining project
Year_To_Net_Zero = 2050 #Time when we want to achieve net zero
Years_To_Mine = Year_To_Net_Zero-Current_Year-Years_To_Develop

#Add in energy costs for industry and waste
#Add in energy costs for industry and waste
Industry_Power = GHG_df.loc[GHG_df['Sector'] == 'Industry', 'Absolute'].values[0]*Power_to_Remove_GHG
Waste_Power = GHG_df.loc[GHG_df['Sector'] == 'Waste', 'Absolute'].values[0]*Power_to_Remove_GHG
power_requirements['Industry'] = Industry_Power
power_requirements['Waste'] = Waste_Power

In [140]:
#Convert Percentiles into values
def Convert_pct_to_Value(pct_array,value):
    #Input an array with pct values with numbers between 0-100 and the value it's going to scale to
    val_array = pct_array/100 * value
    return val_array

def plot_pie(values, labels, title):
    plt.figure(figsize=(10,8))
    plt.pie(values, labels=labels, autopct='%1.1f%%')
    plt.title(title)
    plt.show()

In [132]:
#Direct Carbon Capture
CC_Size = 1.0 #Mt - CO2/y removed
CC_Power = 366 #kWh/t CO2 to compress and store underground
CC_CO2_Removed = World_GHG_Emissions #Total CO2 Emissions Removed per year in Mt
CC_Additional_Power = CC_Power*conversions['kWh_TWh']/conversions['t_Mt']*CC_CO2_Removed #TWh - Will Be Added to Overall Power Chart At End
CC_Stations = CC_CO2_Removed/CC_Size # Number of Stations - Will just be presented

power_requirements['Direct Carbon Capture'] = CC_Additional_Power

In [141]:
#Constants for Vehicle Power Estimates
H2_Drive_Power = 15 #kWh/kg H2
People_per_car = 1.5 # Each car transports on average 1.5 people
People_per_bus = 50 # 1 bus can transport 50 people
E_Bike_to_EV = 1/100 #E-bikes require 100x less material for one unit compared to an EV
EV_Battery_Size = 0.230 #kWh/kg
H2_EV_Power_Conversion = 0.41 #EV vehicles demand ~40% of H2 due to inefficiency in producing H2 (this includes all EV ineffiencies so it's a straight conversion) and is reversible
H2_TWh = 0.017 # Mt H2 / TWh consumed

def Total_Power_Required(df_Vehicles):
    '''
    This calculation is a bit challenging to visualize because there are many different units and efficiency factors if you pick H2 or EV. The values here are designed to align
    with Simon Michaux's work.
    '''
    # Check if all necessary columns are in the DataFrame
    necessary_columns = ['Trillion Km Travelled / y', 'kWh/km Travelled', 'EV Efficiency Overall %', 
                         'Million Vehicles Or Billion (tonne-kilometers) / y', 'TWh Consumed/Million Vehicles/y OR TWh/ Billion tonne - km',
                         'Million Vehicles', 'kWh Batteries / Vehicle']
    for column in necessary_columns:
        if column not in df_Vehicles.columns:
            raise ValueError(f"Column '{column}' not found in the input DataFrame.")

    # Copy the input DataFrame to avoid in-place modifications
    df = df_Vehicles.copy()

    # Calculate the power required for each type of vehicle
    df['Small EV TWh'] = df['Trillion Km Travelled / y']/conversions['km_Tkm']*df['kWh/km Travelled']*conversions['kWh_TWh']/(df['EV Efficiency Overall %']/100)
    df['Small EV TWh'] = df['Small EV TWh'].replace(np.nan, 0)
    df['Small H2 TWh'] = df['Small EV TWh'] / H2_EV_Power_Conversion
    df['Large H2 TWh'] = df['Million Vehicles Or Billion (tonne-kilometers) / y']*df['TWh Consumed/Million Vehicles/y OR TWh/ Billion tonne - km']
    df['Large EV TWh'] = df['Large H2 TWh'] * H2_EV_Power_Conversion

    # Combine Small EV and Large EV into a single column
    df['Total EV TWh'] = df['Small EV TWh'] + df['Large EV TWh']
    df['Total H2 TWh'] = df['Small H2 TWh'] + df['Large H2 TWh']

    # Calculate the total power and material required for the batteries
    df['TW of Motor Power'] = df['Million Vehicles']*conversions['Mv_v'] * df['kWh Batteries / Vehicle']*conversions['kWh_TWh']
    df['EV Mt Batteries'] = df['TW of Motor Power']/EV_Battery_Size*conversions['kWh_TWh']/conversions['kg_t']/conversions['t_Mt']
    df['H2 Mt'] = df['Total H2 TWh']*H2_TWh

    return df

In [142]:
#Determine the total mass of batteries for EV, Mt of H2, and TWh of power required overall
df_Vehicle_Power_and_Mass = Total_Power_Required(df_Vehicles)
#Apply % factors to determine what % of each energy type takes in the power
df_H2_Vehicles = df_Vehicle_Power_and_Mass[['Total H2 TWh','H2 Mt','TW of Motor Power']].multiply(df_Vehicle_Power_and_Mass['% H2']/100, axis="index")
df_EV_Vehicles = df_Vehicle_Power_and_Mass[['Total EV TWh','EV Mt Batteries']].multiply(df_Vehicle_Power_and_Mass['% EV']/100, axis="index")
#Sum the EV power and battery demand after vehicle make up is calculated
Total_EV_Demand_TWh = df_EV_Vehicles['Total EV TWh'].sum()
Total_EV_Battery_Mt = df_EV_Vehicles['EV Mt Batteries'].sum()
#Sum the H2 power and mass demand after vehicle make up is calculated
Total_H2_Demand_TWh = df_H2_Vehicles['Total H2 TWh'].sum()
Total_H2_Demand_Mt = df_H2_Vehicles['H2 Mt'].sum()
Total_H2_Motor_TW = df_H2_Vehicles['TW of Motor Power'].sum()
#Add additional Power to the ongoing dataframe
power_requirements['EV Vehicles'] = Total_EV_Demand_TWh
power_requirements['Hydrogen Production'] = Total_H2_Demand_TWh
Total_power = power_requirements.sum(axis=1)
power_requirements['Total'] = Total_power

In [143]:
#Vehicle Power Estimates
#Multiply in the load demand
df_Metals['EV kt'] = df_Metals['EV %']/100*Total_EV_Battery_Mt*conversions['Mt_kt']
df_Metals['H2 kt'] = df_Metals['H2 Vehicles (g/kWh)']*Total_H2_Motor_TW/conversions['kWh_TWh']*conversions['g_kt']
# print(df_Metals[['Metals','EV kt','H2 kt']])

In [146]:
#Now we plot the total power required in a waterfall chart

import plotly.graph_objects as go

# Preparing data for waterfall chart
waterfall_data = go.Waterfall(
    x = power_requirements.columns.tolist(),
    y = power_requirements.values.flatten().tolist(),
    textposition = "outside",
    text = [f'{v:,.0f}' for v in power_requirements.values.flatten()],
    measure = ["relative", "relative", "relative", "relative", "relative", 'absolute'],
    base = 0,
    increasing = {"marker":{"color":"blue"}},  # Change as required
    decreasing = {"marker":{"color":"red"}},  # Change as required
    totals = {"marker":{"color":"green"}}  # Change as required
)

layout = go.Layout(
    title = "New Renewable Power Make Up Required",
    yaxis = {"title" : "Power Requirement (TWh/y)", 'tickformat': ','},
    margin = dict(l=20, r=20, t=25, b=20), # Modify these values as needed
    height=600,  # adjust as needed
    width=800,  # adjust as needed
)

fig = go.Figure(data=[waterfall_data], layout=layout)
fig.show()

In [147]:
#With total power in mind we can now figure out how much of the power will be taken by each energy source
#Will need a chart for this information
#Input the distribution of each energy type
df_Power_Plants['% New Power'] = [13.9, 26.8, 38.3, 13.4, 7.5,0.0,0.0]

df_Power_Plants['Power TWh/y'] = df_Power_Plants['% New Power']/100*Total_power.sum()
# print(df_Power_Plants[['Type','Power TWh/y']])

#Convert calculations in MW of installed plant power and get the total metal requirement
df_Power_Plants['TWh/plant/year'] = df_Power_Plants['MWh/day/plant']*365*conversions['Mwh_TWh'] #To get total output of a single plant per year
df_Power_Plants['MW/plant effective'] = df_Power_Plants['MWh/day/plant']/24 #By dividing MWh/day by 24h we get MW of installed power which is necessary for metal totals
df_Power_Plants['MW/plant installed'] = df_Power_Plants['MW/plant effective'] / (df_Power_Plants['Capacity Factor']/100) # By applying the capacity factors of each type we get the actualy MW installed which drives metal requirements
df_Power_Plants['kt metal / plant'] = df_Power_Plants['kg metal/ MW']*df_Power_Plants['MW/plant installed']*conversions['kg_t']*conversions['t_kt'] # Get kt of metal per plant based on installed power
df_Power_Plants['Plants Required'] = df_Power_Plants['Power TWh/y']/df_Power_Plants['TWh/plant/year']
df_Power_Plants['kt metal total'] = df_Power_Plants['Plants Required'] * df_Power_Plants['kt metal / plant']

In [ ]:
power_plant_types = ['Offshore wind', 'Onshore wind', 'Solar PV', 'Hydro', 'Nuclear']
percentage_columns = ['Off Shore Wind %', 'On Shore Wind %', 'Solar PV %', 'Hydro %', 'Nuclear %']
new_column_names = ['Offshore Wind kt', 'Onshore Wind kt', 'Solar PV kt', 'Hydro kt', 'Nuclear kt']

for power_plant_type, percentage_column, new_column_name in zip(power_plant_types, percentage_columns, new_column_names):
    metal_total = df_Power_Plants.loc[df_Power_Plants['Type'] == power_plant_type, 'kt metal total'].values[0]
    df_Metals[new_column_name] = df_Metals[percentage_column] / 100 * metal_total

#Need to add uranium metal so here are our constants
U_metal_t_TWh = 22.3 #Tonnes of uranium required for 1 TWh of energy
U_mining_rec = 80 #% recovery of mining
U_mined_t_TWh = U_metal_t_TWh/(U_mining_rec/100) # Effective uranium metal required
Nuclear_Power_TWh = df_Power_Plants['Power TWh/y'].loc[df_Power_Plants['Type'] == 'Nuclear'].values[0] + Power_df['Absolute'].loc[Power_df['Source'] == 'Nuclear'].values[0] #Includes existing nuclear and new nuclear power
U_kt = Nuclear_Power_TWh * U_mined_t_TWh * conversions['t_kt'] #Get total uranium metal required
df_Metals['Nuclear kt'].loc[df_Metals['Metals'] == 'Uranium'] = U_kt

print(df_Metals[['Metals','Nuclear kt']])

In [157]:
#Estimate Nuclear Waste production
Nuclear_waste = {'Waste Type': ['VLLW','LLW','ILW','HLW'],'m3/TWh produced': [29+119,15+186,27+1,0.29]} # m3 / TWh of energy
df_Nuclear_waste = pd.DataFrame(Nuclear_waste)
df_Nuclear_waste['m3 produced / year'] = df_Nuclear_waste['m3/TWh produced'] * Nuclear_Power_TWh
print(df_Nuclear_waste)

  Waste Type  m3/TWh produced  m3 produced / year
0       VLLW           148.00        8.288625e+05
1        LLW           201.00        1.125685e+06
2        ILW            28.00        1.568118e+05
3        HLW             0.29        1.624122e+03


In [168]:
df_Metals.columns

Index(['Metals', 'Current Production (kt/y)', 'Reserve (kt)', 'Resource (kt)',
       'EV %', 'H2 Vehicles (g/kWh)', 'NMC Battery %', 'Lead Battery %',
       'Salt Battery %', 'Off Shore Wind %', 'On Shore Wind %', 'Solar PV %',
       'Hydro %', 'Nuclear %', 'EV kt', 'H2 kt', 'Offshore Wind kt',
       'Onshore Wind kt', 'Solar PV kt', 'Hydro kt', 'Nuclear kt'],
      dtype='object')

In [184]:
# Now we can estimate storage power requirements based on the total Power in Solar and wind systems
# User inputs the proportion of each battery type
Battery_Proportion = {'Battery Type': ['NMC','Lead','Salt'],'% of Total': [75,5,20],'Energy Density kWh/kg': [0.270,0.050,0.160],'Storage Eff %': [84.6,84.6,84.6]} #Values should add up to 100%
df_Battery_Proportion = pd.DataFrame(Battery_Proportion)
#Other Constants
Battery_Facility_Size = 2330 #MWh/facility
Days_of_Storage = 2 # 2 days should be the minimum, while some estimates hold these values closer to 84 days

Renewable_TWh = df_Power_Plants['Power TWh/y'].loc[df_Power_Plants['Type'].isin(['Offshore wind','Onshore wind','Solar PV'])].sum()
#Calculate the to TWh of storage required for each system
df_Battery_Proportion['TWh of Storage'] = df_Battery_Proportion['% of Total']/100*Renewable_TWh/(df_Battery_Proportion['Storage Eff %']/100)*(Days_of_Storage/365)
df_Battery_Proportion['Mt of Batteries'] = df_Battery_Proportion['TWh of Storage']/(df_Battery_Proportion['Energy Density kWh/kg']*conversions['kWh_TWh']/conversions['kg_t']/conversions['t_Mt'])
df_Battery_Proportion['kt of Batteries'] = df_Battery_Proportion['Mt of Batteries']*1000
df_Battery_Proportion['Number of Facilities'] = df_Battery_Proportion['TWh of Storage']/(Battery_Facility_Size*conversions['Mwh_TWh'])

# print(df_Battery_Proportion)

#Map battery production onto Metals dataframe:
battery_types = ['NMC','Lead','Salt']
percentage_columns = ['NMC Battery %', 'Lead Battery %','Salt Battery %']
new_column_names = ['NMC kt', 'Lead kt', 'Salt kt']

for battery_type, percentage_column, new_column_name in zip(battery_types, percentage_columns, new_column_names):
    metal_total = df_Battery_Proportion.loc[df_Battery_Proportion['Battery Type'] == battery_type, 'kt of Batteries'].values[0]
    df_Metals[new_column_name] = df_Metals[percentage_column] / 100 * metal_total

# print(df_Metals[['Metals'] + new_column_names])

In [186]:
#Make just a metals with kt dataframe that we can now use to plot our charts
df_Metals_kt = pd.concat([df_Metals['Metals'], df_Metals.filter(regex='kt')], axis=1)
items = ['EV', 'H2', 'Offshore Wind', 'Onshore Wind', 'Solar PV','Hydro', 'Nuclear', 'NMC', 'Lead', 'Salt']

for item in items:
    df_Metals_kt[item + ' Production kt/y'] = df_Metals_kt[item + ' kt'] / Years_To_Mine

#Here we need a metals charts where the x-axis will be the name of the metal (a single one) and y will the total kt, 
#I want a separate chart of each metal. I want each metal to have 2 bars associated with it. First bar is the combonation of the reserves
#and resources, the second will the a stacked bar chart where we add up each metal (EV, H2, Each Power System, and batteries). Then we need to totalize all the new metals
# (EV, H2, Each Power System, and batteries) and compare that to the current production in kt/y, and this will be shown as lines so one line is current production
# While the second line uses value "Years_To_Mine" to divide the total metal demand by that number of years (base case is 17 years)
# This is the last chart needed
